In [1]:
import pandas as pd
import re
from pathlib import Path
import os
import numpy as np

pd.set_option('display.max_columns', 500)

In [2]:
OUTPUT_DIR = Path("C:/Playground/bhaiya-orders/zoho-bhor-farms/outputs/")

In [3]:
from ipyfilechooser import FileChooser

fc = FileChooser("C:/Playground/bhaiya-orders/zoho-bhor-farms/inputs/")
fc.use_dir_icons = True
display(fc)

FileChooser(path='C:/Playground/bhaiya-orders/zoho-bhor-farms/inputs/', filename='', show_hidden='False')

In [191]:
FILE = Path(fc.selected)

In [192]:
df = pd.read_csv(FILE)

In [193]:
df.head()

,Name,Select Drop Point,Select Building Ivory,Select Building Wellness Forever,Select Building NCP,Select Building Ajmera iLand,Select Building Sector 1/2,Select Building Cosmic,Select Wing,Flat No,Phone,Email,Select Delivery Date,Potatoes,Potato Price,Potato Cost Price,Select Quantity of Potatoes,Red Onions,Red Onions Price,Red Onions Cost Price,Select Quantity of Red Onions,Cauliflower,Cauliflower Price,Cauliflower Cost Price,Select Quantity of Cauliflower,Cabbage,Cabbage Price,Cabbage Cost Price,Select Quantity of Cabbage,Brinjal Bharta,Brinjal Bharta Price,Brinjal Bharta Cost Price,Select Quantity of Brinjal Bharta,Pumpkin,Pumpkin Price,Pumpkin Cost Price,Select Quantity of Pumpkin,Brinjal Kateri,Brinjal Kateri Price,Brinjal Kateri Cost Price,Select Quantity of Brinjal Kateri,Cucumber,Cucumber Price,Cucumber Cost Price,Select Quantity of Cucumber,Bhindi/Lady's Finger,Bhindi/Lady's Finger Price,Bhindi/Lady's Finger Cost Price,Select Quantity of Bhindi/Lady's Finger,Bitter Gourd,Bitter Gourd Price,Bitter Gourd Cost Price,Select Quantity of Bitter Gourd,Sponge Gourd,Sponge Gourd Price,Sponge Gourd Cost Price,Select Quantity of Sponge Gourd,Beetroot,Beetroot Price,Beetroot Cost Price,Select Quantity of Beetroot,Tondli,Tondli Price,Tondli Cost Price,Select Quantity of Tondli,Button Mushrooms,Button Mushroom Price,Button Mushroom Cost Price,Select Quantity of Button Mushroom,Raw Banana,Raw Banana Price,Raw Banana Cost Price,Select Quantity of Raw Banana,Bottle Gourd,Bottle Gourd Price,Bottle Gourd Cost Price,Select Quantity of Bottle Gourd,Drumsticks,Drumsticks Price,Drumsticks Cost Price,Select Quantity of Drumsticks,Capsicum,Capsicum Price,Capsicum Cost Price,Select Quantity of Capsicum,Gawar/Cluster Beans,Gawar/Cluster Beans Price,Gawar/Cluster Beans Cost Price,Select Quantity of Gawar/Cluster Beans,French Beans,French Beans Price,French Beans Cost Price,Select Quantity of French Beans,Baby Corn,Baby Corn Price,Baby Corn Cost Price,Select Quantity of Baby Corn,Tomato,Tomato Price,Tomato Cost Price,Select Quantity of Tomato,Cherry Tomatoes,Cherry Tomato Price,Cherry Tomato Cost Price,Select Quantity of Cherry Tomatoes,Sweet Corn,Sweet Corn Price,Sweet Corn Cost Price,Select Quantity of Sweet Corn,Broccoli,Broccoli Price,Broccoli Cost Price,Select Quantity of Broccoli,Lettuce,Lettuce Price,Lettuce Cost Price,Select Quantity of Lettuce,Apples Washington Per Kg,Apples Washington Price,Apples Washington Cost Price,Select Quantity of Apples Washington,VNR Guava,VNR Guava Price,VNR Guava Cost Price,Select Quantity of VNR Guava,Orange Imported,Imported Oranges Price,Imported Oranges Cost Price,Select Quantity of Imported Oranges,Pineapple Whole,Pineapple Whole Price,Pineapple Whole Cost Price,Select Quantity of Pineapple Whole,Kiwi Zespri Imported,Kiwi Zespri Imported Price,Kiwi Zespri Imported Cost Price,Select Quantity of Kiwi Zespri Imported,Banana Regular,Banana Price,Banana Cost Price,Select Quantity of Banana,Garlic,Garlic Price,Garlic Cost Price,Select Quantity of Garlic,GInger,Ginger Price,Ginger Cost Price,Select Quantity of Ginger,Green Chilly,Green Chilly Price,Green Chilly Cost Price,Select Quantity of Green Chilly,Lemons,Lemons Price,Lemons Cost Price,Select Quantity of Lemons,Curry Leaves,Curry Leaves Price,Curry Leaves Cost Price,Select Quantity of Curry Leaves,Coriander Leaves,Coriander Leaves Price,Coriander Leaves Cost Price,Select Quantity of Coriander Leaves,Mint Leaves,Mint Leaves Price,Mint Leaves Cost Price,Select Quantity of Mint Leaves,Lemongrass,Lemongrass Price,Lemongrass Cost Price,Select Quantity of Lemongrass,Palak/Spinach,Palak/Spinach Price,Palak/Spinach Cost Price,Select Quantity of Palak/Spinach,Shepu/Dill Leaves,Shepu/Dill Leaves Price,Shepu/Dill Leaves Cost Price,Select Quantity of Shepu/Dill Leaves,Methi Leaves,Methi Leaves Price,Methi Leaves Cost Price,Select Quantity of Methi Leaves,Red/Yellow Bell Peppers,Red/Yellow Bell Peppers Price,Red/Yellow Bell Peppers Cost Price,Select Quantity of Red/Ye

## Add indexing key

In [194]:
df['key'] = df['Name'] + ' ' + df['Added Time']

In [195]:
bldg_cols = df.columns[2:8]
df['building'] = df[bldg_cols].fillna('').sum(axis=1)

In [196]:
df.head()

,Name,Select Drop Point,Select Building Ivory,Select Building Wellness Forever,Select Building NCP,Select Building Ajmera iLand,Select Building Sector 1/2,Select Building Cosmic,Select Wing,Flat No,Phone,Email,Select Delivery Date,Potatoes,Potato Price,Potato Cost Price,Select Quantity of Potatoes,Red Onions,Red Onions Price,Red Onions Cost Price,Select Quantity of Red Onions,Cauliflower,Cauliflower Price,Cauliflower Cost Price,Select Quantity of Cauliflower,Cabbage,Cabbage Price,Cabbage Cost Price,Select Quantity of Cabbage,Brinjal Bharta,Brinjal Bharta Price,Brinjal Bharta Cost Price,Select Quantity of Brinjal Bharta,Pumpkin,Pumpkin Price,Pumpkin Cost Price,Select Quantity of Pumpkin,Brinjal Kateri,Brinjal Kateri Price,Brinjal Kateri Cost Price,Select Quantity of Brinjal Kateri,Cucumber,Cucumber Price,Cucumber Cost Price,Select Quantity of Cucumber,Bhindi/Lady's Finger,Bhindi/Lady's Finger Price,Bhindi/Lady's Finger Cost Price,Select Quantity of Bhindi/Lady's Finger,Bitter Gourd,Bitter Gourd Price,Bitter Gourd Cost Price,Select Quantity of Bitter Gourd,Sponge Gourd,Sponge Gourd Price,Sponge Gourd Cost Price,Select Quantity of Sponge Gourd,Beetroot,Beetroot Price,Beetroot Cost Price,Select Quantity of Beetroot,Tondli,Tondli Price,Tondli Cost Price,Select Quantity of Tondli,Button Mushrooms,Button Mushroom Price,Button Mushroom Cost Price,Select Quantity of Button Mushroom,Raw Banana,Raw Banana Price,Raw Banana Cost Price,Select Quantity of Raw Banana,Bottle Gourd,Bottle Gourd Price,Bottle Gourd Cost Price,Select Quantity of Bottle Gourd,Drumsticks,Drumsticks Price,Drumsticks Cost Price,Select Quantity of Drumsticks,Capsicum,Capsicum Price,Capsicum Cost Price,Select Quantity of Capsicum,Gawar/Cluster Beans,Gawar/Cluster Beans Price,Gawar/Cluster Beans Cost Price,Select Quantity of Gawar/Cluster Beans,French Beans,French Beans Price,French Beans Cost Price,Select Quantity of French Beans,Baby Corn,Baby Corn Price,Baby Corn Cost Price,Select Quantity of Baby Corn,Tomato,Tomato Price,Tomato Cost Price,Select Quantity of Tomato,Cherry Tomatoes,Cherry Tomato Price,Cherry Tomato Cost Price,Select Quantity of Cherry Tomatoes,Sweet Corn,Sweet Corn Price,Sweet Corn Cost Price,Select Quantity of Sweet Corn,Broccoli,Broccoli Price,Broccoli Cost Price,Select Quantity of Broccoli,Lettuce,Lettuce Price,Lettuce Cost Price,Select Quantity of Lettuce,Apples Washington Per Kg,Apples Washington Price,Apples Washington Cost Price,Select Quantity of Apples Washington,VNR Guava,VNR Guava Price,VNR Guava Cost Price,Select Quantity of VNR Guava,Orange Imported,Imported Oranges Price,Imported Oranges Cost Price,Select Quantity of Imported Oranges,Pineapple Whole,Pineapple Whole Price,Pineapple Whole Cost Price,Select Quantity of Pineapple Whole,Kiwi Zespri Imported,Kiwi Zespri Imported Price,Kiwi Zespri Imported Cost Price,Select Quantity of Kiwi Zespri Imported,Banana Regular,Banana Price,Banana Cost Price,Select Quantity of Banana,Garlic,Garlic Price,Garlic Cost Price,Select Quantity of Garlic,GInger,Ginger Price,Ginger Cost Price,Select Quantity of Ginger,Green Chilly,Green Chilly Price,Green Chilly Cost Price,Select Quantity of Green Chilly,Lemons,Lemons Price,Lemons Cost Price,Select Quantity of Lemons,Curry Leaves,Curry Leaves Price,Curry Leaves Cost Price,Select Quantity of Curry Leaves,Coriander Leaves,Coriander Leaves Price,Coriander Leaves Cost Price,Select Quantity of Coriander Leaves,Mint Leaves,Mint Leaves Price,Mint Leaves Cost Price,Select Quantity of Mint Leaves,Lemongrass,Lemongrass Price,Lemongrass Cost Price,Select Quantity of Lemongrass,Palak/Spinach,Palak/Spinach Price,Palak/Spinach Cost Price,Select Quantity of Palak/Spinach,Shepu/Dill Leaves,Shepu/Dill Leaves Price,Shepu/Dill Leaves Cost Price,Select Quantity of Shepu/Dill Leaves,Methi Leaves,Methi Leaves Price,Methi Leaves Cost Price,Select Quantity of Methi Leaves,Red/Yellow Bell Peppers,Red/Yellow Bell Peppers Price,Red/Yellow Bell Peppers Cost Price,Select Quantity of Red/Ye

In [197]:
df['Payment Status'].unique()

array(['Completed', 'Processing not needed', 'Failed'], dtype=object)

## Subset only successfully paid orders

In [198]:
df = df[df['Payment Status'].isin(['Completed', 'Network Error'])]

In [199]:
df['Payment Status'].unique()

array(['Completed'], dtype=object)

## Generating Tokens

In [200]:
df = df.sort_values(['Select Drop Point', 'building', 'Select Wing'])
df['Token No'] = range(1, len(df)+1)

## Check key uniqueness

In [201]:
assert len(df) == len(df['key'].unique())

## Build dictionary

    orders dictionary, one record looks like:

    {
        key:
        {
            name : str,
            drop_point : str,
            building : str,
            wing : str,
            flat : str,
            phone : numeric,
            email : str,
            delivery_date : datetime,
            order : dict,
            total_amount : float,
            payment_status : str
        }
    }


    items_dict:

    {
        item_name : 
        {
            sell_price : float,
            cost_price : float,
            packing_size : // to be read from another excel
        }
    }

In [202]:
# # ask bhaiya
# alt_all_items = []
# for i, col in enumerate(df.columns):
#     if 'select quantity of' in col.lower():
#         alt_all_items.append(df.columns[i-3])

In [203]:
# extremely naive, needs a fix
all_items = [i[19:] for i in df.columns if 'quantity of' in i.lower()]

In [204]:
for i in all_items:
    print(i)

Potatoes
Red Onions
Cauliflower
Cabbage
Brinjal Bharta
Pumpkin
Brinjal Kateri
Cucumber
Bhindi/Lady's Finger
Bitter Gourd
Sponge Gourd
Beetroot
Tondli
Button Mushroom
Raw Banana
Bottle Gourd
Drumsticks
Capsicum
Gawar/Cluster Beans
French Beans
Baby Corn
Tomato
Cherry Tomatoes
Sweet Corn
Broccoli
Lettuce
Apples Washington
VNR Guava
Imported Oranges
Pineapple Whole
Kiwi Zespri Imported
Banana
Garlic
Ginger
Green Chilly
Lemons
Curry Leaves
Coriander Leaves
Mint Leaves
Lemongrass
Palak/Spinach
Shepu/Dill Leaves
Methi Leaves
Red/Yellow Bell Peppers
Green Zucchini
Yellow Zucchini
Arbi Leaves
Basil Leaves
Arbi/Colocasia


In [205]:
items_dict = {}

In [207]:
def get_sell_price(item):
    val = df[item + ' ' + 'Price'].values[0]
    return val

def get_cost_price(item):
    val = df[item + ' ' + 'Cost Price'].values[0]
    return val

In [208]:
df_psize = pd.read_csv("C:/Playground/bhaiya-orders/zoho-bhor-farms/database/Packing Size.csv")
psize_dict = df_psize.set_index('Item').to_dict(orient='index')

In [209]:
def get_packing_size(item):
    return psize_dict[item]['Size']

def get_unit(item):
    return psize_dict[item]['Unit']

In [210]:
for item in all_items:
    dct = {}
    dct['packing_size'] = get_packing_size(item)
    dct['unit'] = get_unit(item)
    if item == 'Cherry Tomatoes': item = 'Cherry Tomato'
    if item == 'Potatoes': item = 'Potato'
    
    dct['sell_price'] = get_sell_price(item)
    dct['cost_price'] = get_cost_price(item)
    items_dict[item] = dct

In [211]:
psize_dict

{'Amla': {'Size': 0.5, 'Unit': 'Kg'},
 'Apples New Zealand': {'Size': 1.0, 'Unit': 'Kg'},
 'Apples Washington': {'Size': 1.0, 'Unit': 'Kg'},
 'Arbi Leaves': {'Size': 6.0, 'Unit': 'Pieces'},
 'Arbi/Colocasia': {'Size': 0.5, 'Unit': 'Kg'},
 'Assorted Flowers Large': {'Size': 1.0, 'Unit': 'Kg'},
 'Assorted Mango Box': {'Size': 2.0, 'Unit': 'Kg'},
 'Baby Corn': {'Size': 0.3, 'Unit': 'Kg'},
 'Banana': {'Size': 6.0, 'Unit': 'Pieces'},
 'Basil Leaves': {'Size': 0.2, 'Unit': 'Kg'},
 'Beetroot': {'Size': 0.5, 'Unit': 'Kg'},
 "Bhindi/Lady's Finger": {'Size': 0.5, 'Unit': 'Kg'},
 'Bitter Gourd': {'Size': 0.3, 'Unit': 'Kg'},
 'Bottle Gourd': {'Size': 1.0, 'Unit': 'Pieces'},
 'Brinjal Bharta': {'Size': 1.0, 'Unit': 'Pieces'},
 'Brinjal Kateri': {'Size': 0.5, 'Unit': 'Kg'},
 'Broccoli': {'Size': 0.25, 'Unit': 'Kg'},
 'Button Mushroom': {'Size': 0.2, 'Unit': 'Kg'},
 'Cabbage': {'Size': 1.0, 'Unit': 'Pieces'},
 'Capsicum': {'Size': 0.5, 'Unit': 'Kg'},
 'Cauliflower': {'Size': 1.0, 'Unit': 'Pieces'},
 

## Build Master Dictionary

In [212]:
bldg_cols = df.columns[2:8]
df['building'] = df[bldg_cols].fillna('').sum(axis=1)

In [213]:
# because few flats were not captured
df['Flat No'] = df['Flat No'].fillna('NOT CAPTURED')

In [214]:
df.fillna(0, inplace=True)

In [215]:
# create a mapping from item_id to the corresponding column to make sure no item goes unmapped

item2dropdowncol = {i : '' for i in all_items}

for index, row in df.iterrows():
    for col in df.columns:
        if 'quantity of' in col.lower():
            item2dropdowncol[col[19:]] = col
            
assert len(item2dropdowncol) == len(all_items)
for k, v in item2dropdowncol.items():
    assert len(v) > 3


    {
        key:
        {
            name : str,
            drop_point : str,
            building : str,
            wing : str,
            flat : str,
            phone : numeric,
            email : str,
            delivery_date : datetime,
            order : dict,
            total_amount : float,
            payment_status : str
        }
    }

In [216]:
def get_order(row):
    o = {}
    for item, qty_col in item2dropdowncol.items():
        if row[qty_col] > 0:
            o[item] = row[qty_col]
            
    return o

In [217]:
orders_dict = {}

for index, row in df.iterrows():
    dct = {}
    dct['name'] = row['Name']
    dct['drop_point'] = row['Select Drop Point']
    dct['building'] = row['building']
    dct['wing'] = row['Select Wing']
    dct['flat_no'] = row['Flat No']
    dct['phone_no'] = row['Phone']
    dct['email'] = row['Email']
    dct['delivery_date'] = row['Select Delivery Date']
    dct['total_amount'] = row['Total Amount']
    dct['payment_status'] = row['Payment Status']
    dct['order'] = get_order(row)
    
    orders_dict[row['Token No']] = dct

## Create Listified File

In [218]:
dict_listified = {i : 0 for i in all_items}

In [219]:
for key, value in orders_dict.items():
    for item, qty in value['order'].items():
        dict_listified[item] += qty

In [220]:
df_listified = pd.DataFrame()

df_listified['item'] = all_items
df_listified['packing_size'] = df_listified['item'].apply(lambda x: psize_dict[x]['Size'])
df_listified['num_units'] = df_listified['item'].apply(lambda x: dict_listified[x])
df_listified['weight'] = df_listified.packing_size * df_listified.num_units
df_listified['unit'] = df_listified['item'].apply(lambda x: psize_dict[x]['Unit'])

In [221]:
#TODO handle zeros

In [222]:
df_listified

,item,packing_size,num_units,weight,unit
0,Potatoes,1.00,78.0,78.00,Kg
1,Red Onions,1.00,87.0,87.00,Kg
2,Cauliflower,1.00,55.0,55.00,Pieces
3,Cabbage,1.00,33.0,33.00,Pieces
4,Brinjal Bharta,1.00,48.0,48.00,Pieces
5,Pumpkin,0.50,32.0,16.00,Kg
6,Brinjal Kateri,0.50,13.0,6.50,Kg
7,Cucumber,0.50,62.0,31.00,Kg
8,Bhindi/Lady's Finger,0.50,60.0,30.00,Kg
9,Bitter Gourd,0.30,26.0,7.80,Kg


In [223]:
save_pth = Path('output/' + str(FILE.stem)); save_pth
os.makedirs(save_pth, exist_ok=True)

In [224]:
fname = save_pth/('quantity-' + str(FILE.stem) + '.csv')

In [225]:
df_listified.to_csv(fname, index=False)

## Customerwise

In [226]:
items = ['coconut', 'kiwi']

In [227]:
df_trans = pd.read_excel("C:/Playground/bhaiya-orders/zoho-bhor-farms/database/translations.xlsx")

In [228]:
from fuzzywuzzy import fuzz

def get_match(x, thresh=65):
    x = x.lower()
    rmax = -1
    for k in items: 
        if fuzz.partial_ratio(k, x) > rmax:
            rmax = fuzz.partial_ratio(k, x)

    return rmax > thresh

In [229]:
keys2drop = df_trans[df_trans['Final Item Name'].apply(get_match)]['Final Item Name'].values; keys2drop

array(['Coconut', 'Kiwi Imported', 'Coconut Mangalore Small',
       'Kiwi Imported', ' Coconut Mangalore Big',
       ' Coconut Mangalore Medium', 'Kiwi Imported - Zespri Green'],
      dtype=object)

In [230]:
df_trans = df_trans.set_index('Final Item Name').drop(keys2drop).reset_index()
eng2marathi = df_trans.set_index('Final Item Name').to_dict()['Final Marathi Name']
marathi2eng = df_trans.set_index('Final Marathi Name').to_dict()['Final Item Name']
marathi2eng[np.nan] = ''

In [231]:
def get_trans(x, thresh=85):
    rmax = -1
    trans = ''
    for k in eng2marathi.keys():
        if fuzz.ratio(k, x) > rmax:
            rmax = fuzz.ratio(k, x)
            trans = eng2marathi[k]

    if rmax > thresh:
        return trans
    return np.nan

In [232]:
rows_list = []
for token, o in orders_dict.items():
    for item, qty in o['order'].items():
        row_dct = {}
        row_dct['token_no'] = token
        row_dct['name'] = o['name']
        row_dct['delivery_date'] = o['delivery_date']
        row_dct['item_name'] = item
        row_dct['marathi_name'] = get_trans(item)
        row_dct['num_packets'] = qty
        row_dct['packing_size'] = psize_dict[item]['Size']
        row_dct['final_weight'] = row_dct['num_packets'] * row_dct['packing_size']
        row_dct['unit'] = psize_dict[item]['Unit']
        row_dct['total_amt'] = o['total_amount']
        
        rows_list.append(row_dct)

In [233]:
df_customerwise = pd.DataFrame(rows_list)

In [234]:
df_customerwise.head()

,token_no,name,delivery_date,item_name,marathi_name,num_packets,packing_size,final_weight,unit,total_amt
0,1,Kumkum; Jain,Thursday Aug 20th,Cabbage,कोबी,1.0,1.0,1.0,Pieces,449
1,1,Kumkum; Jain,Thursday Aug 20th,Beetroot,बीटरूट,2.0,0.5,1.0,Kg,449
2,1,Kumkum; Jain,Thursday Aug 20th,Bottle Gourd,दुध्या भोपळा,2.0,1.0,2.0,Pieces,449
3,1,Kumkum; Jain,Thursday Aug 20th,Capsicum,शिमला मिर्ची,1.0,0.5,0.5,Kg,449
4,1,Kumkum; Jain,Thursday Aug 20th,Tomato,टोमॅटो,2.0,1.0,2.0,Kg,449


In [235]:
notranslist = df_customerwise[df_customerwise.marathi_name.isna()].item_name.unique(); notranslist

array(['Kiwi Zespri Imported'], dtype=object)

## Kiwi alag se

In [236]:
save_pth = Path("C:/Playground/bhaiya-orders/zoho-bhor-farms/testop")

In [237]:
ids = set([i.lower()[:4] for i in notranslist])
prefix = "".join(ids)

unavl_pth = save_pth/f'{prefix}-{str(FILE.stem)}.csv'; unavl_pth.stem

'kiwi-Final Complete PAyment'

In [238]:
df_customerwise[df_customerwise.marathi_name.isna()].drop(columns=['marathi_name', 'total_amt']).to_csv(unavl_pth, index=False)

## Drop Unavl

In [239]:
df_customerwise.dropna(subset=['marathi_name'], inplace=True)

## Build Savefile

In [240]:
highlight_col = 'num_packets'

In [241]:
custs = df_customerwise.name + '_' + df_customerwise.token_no.astype(str)
custs = custs.unique()

In [242]:
def f(x):
    col = highlight_col
    r = 'background-color: none'
    g = 'background-color: #37FDFC'
    c = np.where(x[col] > 1, g, r)
    y = pd.DataFrame('', index=x.index, columns=x.columns)
    for h in all_cols:
        y[h] = c
    
    return y

In [243]:
def boldify(x):
    col = 'total_amt'
    n = 'font-weight: normal'
    b = 'font-weight: bold'
    c = np.where(x[col] >= 1000.0, b, n)
    y = pd.DataFrame('', index=x.index, columns=x.columns)
    for h in all_cols:
        y[h] = c
    
    return y

In [244]:
all_cols = df_customerwise.columns

In [245]:
def get_sum_packets(token):
    return sum(orders_dict[token]['order'].values())

In [246]:
fname = save_pth/('customerwise-' + str(FILE.stem) + '.xlsx')

In [247]:
#TODO add all borders

In [248]:
pdfy_data = data.to_numpy().tolist()
pdfy_data.insert(0, data.columns.values.tolist())

In [249]:
filename = 'yedekhopdf.pdf'

In [250]:
from reportlab.platypus import SimpleDocTemplate
from reportlab.lib.pagesizes import letter, landscape

In [251]:
from reportlab.pdfbase.ttfonts import TTFont  

In [252]:
from reportlab.pdfbase import pdfmetrics

In [253]:
pdfmetrics.registerFont(TTFont("mangal", "font/Lohit-Devanagari.ttf"))

In [254]:
pdf = SimpleDocTemplate(
    filename,
    pagesize=landscape(letter),
#     font='mangal'
)

In [255]:
from reportlab.platypus import Table
table = Table(pdfy_data)

In [256]:
from reportlab.platypus import TableStyle
from reportlab.lib import colors

In [257]:
style = TableStyle([
#     ('BACKGROUND', (0,0), (3,0), colors.green),
#     ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),

    ('ALIGN',(0,0),(-1,-1),'CENTER'),
#     ('FONTNAME', (0,0), (-1,0), 'Courier-Bold'),
    ('FONTSIZE', (0,0), (-1,0), 10),
    ('BOTTOMPADDING', (0,0), (-1,0), 12),
    ('FONTNAME',(4,1),(4,-1),'mangal'),
    ('BACKGROUND',(0,1),(-1,-1),colors.beige),
])
table.setStyle(style)

In [329]:
rowNumb = len(pdfy_data)
for i in range(1, rowNumb):
    if pdfy_data[i][-5] > 1:
        bc = colors.burlywood
    else:
        bc = colors.white
    ts = TableStyle(
        [('BACKGROUND', (0,i),(-1,i), bc)]
    )
    table.setStyle(ts)

In [330]:
# 3) Add borders
ts = TableStyle(
    [
#     ('BOX',(0,0),(-1,-1),2,colors.black),

#     ('LINEBEFORE',(2,1),(2,-1),2,colors.red),
#     ('LINEABOVE',(0,2),(-1,2),2,colors.green),

    ('GRID',(0,0),(-1,-1),2,colors.black),
    ]
)
table.setStyle(ts)

In [331]:
elems = []
elems.append(table)

In [332]:
pdf.build(elems)

In [156]:
fname

WindowsPath('C:/Playground/bhaiya-orders/zoho-bhor-farms/testop/customerwise-MangoesandOtherSeasonalFruitsForm17thAug2020_Report.xlsx')

In [258]:
for cust in custs:
    data = df_customerwise[df_customerwise.name == cust.split('_')[0]]
    data = data[data.token_no == int(cust.split('_')[1])]
    
    break

In [259]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for cust in custs:
        data = df_customerwise[df_customerwise.name == cust.split('_')[0]]
        data = data[data.token_no == int(cust.split('_')[1])]
        all_data += len(data.index)
        
        data = data.append({'marathi_name': 'TOTAL PACKETS', 'num_packets': get_sum_packets(int(cust.split('_')[1]))}, ignore_index=True)
        styled = data.style.apply(f, axis=None)
        styled = styled.apply(boldify, axis=None)
        styled.to_excel(writer, columns=['token_no', 'name', 'delivery_date', 'item_name', 'marathi_name',
       'num_packets', 'packing_size', 'final_weight', 'unit'], sheet_name=cust[:31], engine='openpyxl', index=False)
        
assert all_data == len(df_customerwise)

## Printout File

In [260]:
# decision for vegetables vs seasonal fruits

In [261]:
def is_trans(x, thresh=85):
    rmax = -1
    trans = ''
    for k in eng2marathi.keys():
        if fuzz.ratio(k, x) > rmax:
            rmax = fuzz.ratio(k, x)
            trans = eng2marathi[k]

    return rmax > thresh

In [262]:
translations = [is_trans(i) for i in all_items]

In [263]:
pipe_separated = sum(translations)/len(translations) < 0.4 # less than 40% translations are present

In [264]:
pipe_separated

False

### Sudeep

In [265]:
def get_pipe_separated(token):
    order = orders_dict[token]['order']
    s=[]
    for k, v in order.items():
        s.append(k + ': ' + str(int(v)))
    
    return ' | '.join(s)

In [266]:
rows_list_sudeep = []
for token, o in orders_dict.items():
    row_dct = {}
    row_dct['token_no'] = token
    row_dct['name'] = o['name']
    row_dct['drop_point'] = o['drop_point']
    row_dct['building'] = o['building']
    row_dct['wing'] = o['wing']
    row_dct['flat_no'] = o['flat_no']
    row_dct['phone_no'] = o['phone_no']
    row_dct['email'] = o['email']
    
    if pipe_separated: row_dct['items'] = get_pipe_separated(token)
    else: row_dct['num_packets'] = get_sum_packets(token)
    
    row_dct['total_amount'] = o['total_amount']
    
    rows_list_sudeep.append(row_dct)

In [267]:
df_print_sudeep = pd.DataFrame(rows_list_sudeep)

In [268]:
df_print_sudeep.head()

,token_no,name,drop_point,building,wing,flat_no,phone_no,email,num_packets,total_amount
0,1,Kumkum; Jain,Ajmera iLand,Aeon,A Wing,302,9.930161e+09,Jainkmkm@gmail.com,16.0,449
1,2,Anuradha; Singh,Ajmera iLand,Zeon,B Wing,2704,9.167165e+09,anuradha19.1.71@gmail.com,20.0,711
2,3,Juhi; Sethi,Ajmera iLand,Zeon,B Wing,901,9.967052e+09,Aqua1379@gmail.com,23.0,948
3,4,Bharti; Morey,Cosmic,Comet,C Wing,101,9.819985e+09,bharti@morey.in,13.0,359
4,5,Lynet; Fernandes,Cosmic,Comet,C Wing,203,9.821142e+09,Fernandeslynet@gmail.com,11.0,354


In [269]:
save_pth

WindowsPath('C:/Playground/bhaiya-orders/zoho-bhor-farms/testop')

In [270]:
df_print_sudeep.to_csv(save_pth/'sabji-sudeep-print.csv', index=False)

### Vishal

In [271]:
rows_list_vishal = []
for token, o in orders_dict.items():
    row_dct = {}
    row_dct['token_no'] = token
    row_dct['name'] = o['name']
    row_dct['drop_point'] = o['drop_point']
    row_dct['building'] = o['building']
    row_dct['wing'] = o['wing']
    row_dct['flat_no'] = o['flat_no']
#     row_dct['phone_no'] = o['phone_no']
#     row_dct['email'] = o['email']

    if pipe_separated: row_dct['items'] = get_pipe_separated(token)
    else: row_dct['num_packets'] = get_sum_packets(token)    
    
    row_dct['total_amount'] = o['total_amount']
    
    rows_list_vishal.append(row_dct)

In [272]:
df_print_vishal = pd.DataFrame(rows_list_vishal)

In [273]:
df_print_vishal.head()

,token_no,name,drop_point,building,wing,flat_no,num_packets,total_amount
0,1,Kumkum; Jain,Ajmera iLand,Aeon,A Wing,302,16.0,449
1,2,Anuradha; Singh,Ajmera iLand,Zeon,B Wing,2704,20.0,711
2,3,Juhi; Sethi,Ajmera iLand,Zeon,B Wing,901,23.0,948
3,4,Bharti; Morey,Cosmic,Comet,C Wing,101,13.0,359
4,5,Lynet; Fernandes,Cosmic,Comet,C Wing,203,11.0,354


In [274]:
df_print_vishal.to_csv(save_pth/'sabji-vishal-print.csv', index=False)

### Dropwise 

In [275]:
drops = df_print_vishal.drop_point.unique()

In [276]:
fname = save_pth/"sabji-dropwise.xlsx"

In [277]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for drop in drops:
        data = df_print_vishal[df_print_vishal.drop_point == drop]
#         data = data[data.token_no == int(cust.split('_')[1])]
        all_data += len(data.index)
        
#         data = data.append({'marathi_name': 'TOTAL PACKETS', 'num_packets': get_sum_packets(int(cust.split('_')[1]))}, ignore_index=True)
#         styled = data.style.apply(f, axis=None)
#         styled = styled.apply(boldify, axis=None)
#         styled.to_excel(writer, columns=['token_no', 'name', 'delivery_date', 'item_name', 'marathi_name',
#        'num_packets', 'packing_size', 'final_weight', 'unit'], sheet_name=cust[:31], engine='openpyxl', index=False)
        
        invalid_chars = '[]:*?/\\'
        for c in invalid_chars: drop = drop.replace(c, '')
        
        data.to_excel(writer, sheet_name=drop, index=False)
    
assert all_data == len(df_print_vishal)

### Tokens

In [278]:
subset_cols = ['name', 'token_no', 'building', 'wing', 'flat_no']

In [279]:
df_tokens = df_print_vishal[subset_cols]

In [280]:
drops = df_print_vishal.drop_point.unique()

In [281]:
fname = save_pth/"sabji-tokens.xlsx"

In [282]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for drop in drops:
        data = df_print_vishal[df_print_vishal.drop_point == drop]
#         data = data[data.token_no == int(cust.split('_')[1])]
        all_data += len(data.index)
        
#         data = data.append({'marathi_name': 'TOTAL PACKETS', 'num_packets': get_sum_packets(int(cust.split('_')[1]))}, ignore_index=True)
#         styled = data.style.apply(f, axis=None)
#         styled = styled.apply(boldify, axis=None)
#         styled.to_excel(writer, columns=['token_no', 'name', 'delivery_date', 'item_name', 'marathi_name',
#        'num_packets', 'packing_size', 'final_weight', 'unit'], sheet_name=cust[:31], engine='openpyxl', index=False)
        
        invalid_chars = '[]:*?/\\'
        for c in invalid_chars: drop = drop.replace(c, '')
        
        data.to_excel(writer, sheet_name=drop, columns=subset_cols, index=False)
    
assert all_data == len(df_print_vishal)

In [283]:
df_print_sudeep.to_csv('printout-sitaphal.csv', index=False)

In [ ]:
def get_pipe_sep_orders(token):

In [ ]:
rows_list = []
for token, o in orders_dict.items():
    row_dct = {}
    row_dct['token_no'] = token
    row_dct['name'] = o['name']
    row_dct['drop_point'] = o['drop_point']
    row_dct['building'] = o['building']
    row_dct['wing'] = o['wing']
    row_dct['flat_no'] = o['flat_no']
    row_dct['phone_no'] = o['phone_no']
    row_dct['email'] = o['email']
    row_dct['num_packets'] = get_sum_packets(token)
    row_dct['total_amount'] = o['total_amount']
    
    rows_list.append(row_dct)

## Kachcha Dropwise

In [183]:
orders_dict

{1: {'name': 'Rashmita; Sundaram',
  'drop_point': 'Ajmera iLand',
  'building': 'Aeon',
  'wing': 'A Wing',
  'flat_no': '1202',
  'phone_no': 9004913659.0,
  'email': 'Rashmita.sundaram@gmail.com',
  'delivery_date': 'Wednesday Aug 19th',
  'total_amount': 274,
  'payment_status': 'Completed',
  'order': {'VNR Guava': 1.0, 'Elaichi Banana Regular': 1.0, 'Amla': 1.0}},
 2: {'name': 'Kumkum; Jain',
  'drop_point': 'Ajmera iLand',
  'building': 'Aeon',
  'wing': 'A Wing',
  'flat_no': '302',
  'phone_no': 9930160637.0,
  'email': 'Jainkmkm@gmail.com',
  'delivery_date': 'Wednesday Aug 19th',
  'total_amount': 535,
  'payment_status': 'Completed',
  'order': {'Elaichi Banana Regular': 1.0,
   'Langra Mangoes': 1.0,
   'Papaya': 2.0}},
 3: {'name': 'Prerna; Jain',
  'drop_point': 'Ajmera iLand',
  'building': 'Zeon',
  'wing': 'A Wing',
  'flat_no': '2403',
  'phone_no': 9920579019.0,
  'email': 'prerna206@gmail.com',
  'delivery_date': 'Wednesday Aug 19th',
  'total_amount': 730,
  'paym

In [185]:
rows_list = []
for token, order in orders_dict.items():
    for item, count in order['order'].items():
        row_dct = {}
        row_dct['drop_point'] = order['drop_point']
        row_dct['item_name'] = item
        row_dct['num_packets'] = count
        
        rows_list.append(row_dct)

In [186]:
df_dropwise = pd.DataFrame(rows_list)

In [187]:
df_dropwise

,drop_point,item_name,num_packets
0,Ajmera iLand,VNR Guava,1.0
1,Ajmera iLand,Elaichi Banana Regular,1.0
2,Ajmera iLand,Amla,1.0
3,Ajmera iLand,Elaichi Banana Regular,1.0
4,Ajmera iLand,Langra Mangoes,1.0
...,...,...,...
181,Sector 1/2,Pomegranate Regular,1.0
182,Sector 1/2,Premium Long Raisins,2.0
183,Sector 1/2,Elaichi Banana Premium,1.0
184,Sector 1/2,Watermelon,1.0


In [188]:
df_dropwise.groupby(['drop_point', 'item_name']).sum()

num_packets
drop_point   item_name                          
Ajmera iLand Amla                            1.0
             Apples New Zealand              1.0
             Assorted Mango Box              1.0
             Chausa Mangoes                  1.0
             Elaichi Banana Premium          1.0
...                                          ...
Sector 1/2   Papaya                          1.0
             Pomegranate Regular             1.0
             Premium Long Raisins            3.0
             VNR Guava                       2.0
             Watermelon                      2.0

[73 rows x 1 columns]

In [190]:
df_dropwise.groupby(['drop_point', 'item_name']).sum().reset_index().to_csv('asli-dropwise-mango.csv', index=False)